# Assignment 5 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [1]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [2]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [3]:
# word net installation:

# unmark if you want to use and need to install
# !pip install wn
# !python -m wn download omw-he:1.4

In [4]:
# word net import:

# unmark if you want to use:
# import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [5]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
# !pip install hebrew_tokenizer

In [6]:
# Hebrew tokenizer import:

# unmark if you want to use:
# import hebrew_tokenizer

### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [7]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [8]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [9]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

### This exercise done by:

Matan Kichler  
ID 318850427

In [10]:
def preprocessing(df: pd.DataFrame):
    # Removing non Hebrew charecters
    df['story'] = df['story'].str.replace(r'[^\u0590-\u05fe\s]+', '', regex=True)

    # STEEMING:
        # Remove prefixes : 'כ' 'ו' 'ב' 'ש' 'ה' 'ל' 'כש' 'לכש' 'וש' 'וכש'
    df['story'] = df['story'].apply(lambda x: re.sub(r'\b(כ|ו|כש|וכש|וש|לכש|ב|ש|ה|ל)', '', x))
        # Remove suffixes : 'תי' 'נו' 'ים' 'ו'
    df['story'] = df['story'].apply(lambda x: re.sub(r'(תי|ו|נו|ים)\b', '', x))
    df['story'] = df['story'].apply(lambda x: re.sub(r'ות\b', 'ה', x)) # בחורות -> בחורה
    
        # fixing words - converting to final form
    df['story'] = df['story'].apply(lambda x: re.sub(r'מ\b', 'ם', x))
    df['story'] = df['story'].apply(lambda x: re.sub(r'נ\b', 'ן', x))
    df['story'] = df['story'].apply(lambda x: re.sub(r'פ\b', 'ף', x))
    df['story'] = df['story'].apply(lambda x: re.sub(r'כ\b', 'ך', x))
    df['story'] = df['story'].apply(lambda x: re.sub(r'צ\b', 'ץ', x))
    
    # Removing words that has less than 3 letters - many of them are stop words
    df['story'] = df['story'].str.replace(r'\b\w{1,2}\b', '', regex=True)     
    
    # Removing multiply spaces
    df['story'] = df['story'].apply(lambda x: re.sub(r'\s+', ' ', x))


In [11]:
# YOUR CODE HERE
preprocessing(df_train)
preprocessing(df_test)

In [12]:
df_train['story'][2]

'מאז התחיל לימוד חלום סטודנט פנגויה מחפש חבר קובע קבוצה הכיר אנש חדש מכללה מיוחד מכיר אחד מראש אבל אחרי מאמץ צלח מצוא עצמי חברה אנש נחמד היה איתם כבר חודש ראשון תואר ידע אני נוסעת חברה חדשה פגש כיתה רצי לכת יחד ראש עובדה חלק רווקה חלק זוגיה אבל הכי רצי עשה לכת חגוג ישר קבע יחד אפיל חדר סגר שהיי חול אצל קוח חברה אכפת פנה זמן נוכל לכת להנה תחילה קורונה שיהשה תוכניה יצא במשך תקופה מאוד ארוכה קורונה פסקה מספר פעם במקום מאי אותה פנגויה יתה ספטמבר נתי אחרי ינוי מסיבה בתוכניה פנגויה לומר פנגויה רגילה אבל אחרי נתי אחד אכפת אחת מהבנה יתה סטטוס אחר יצא במקום נשאר אבל פחה חברה טובה משך תקופה מאוד ארוכה תיכן נקנה נלבש לאן נצא אפיל קבע מקום מסעדה נקראת מחבוא אדי אנש חודש מראש גיע שבוע פני פנגויה התרגשה שיא אבל אני הרגשה התחלה צינון אחרי יעול חום בסוף חלט עשה דיקת קורונה הבין קורה קיבל תשובה חיובית תבאס רמה נכנס בידוד הדביק אחד יצא אחרי נתי למה מתנה אני צריכה המתין עוד למה פנגויה באה קיבל תמונה סרטון קרע הוציא עיני חוש טעם ריח משפיע עלי יום קיצור מסקנה מהסיפור דבר ראשון צפה יותר מידי דבר דבר התחסן ק

In [13]:
def get_tokens(text):
    return text.split(' ')

def vectorizer(df_train, df_test):
    # max_df=400 removes many of the stop words
    # min_df=4 removes some typos and rear words
    vect = CountVectorizer(tokenizer=get_tokens, ngram_range=(1,3), min_df=5, max_df=400, max_features=10000) 
    
    df = pd.concat([df_train, df_test], axis=0)
    
    X = vect.fit_transform( df['story'].to_list() )
    y = df_train[df_train.columns[-1]].replace({'m': 0, 'f': 1}) # mapping 'm' to 0 and 'f' to 1

    display(pd.DataFrame(X.toarray(), columns=vect.get_feature_names_out()).iloc[:, 100:121])
    
    return X[:len(df_train)], X[len(df_train):], y

In [14]:
X_train, X_test, y_train = vectorizer(df_train, df_test)

,אהבת,אהוב,אהוב עלי,אהוב עליי,אהובה,אהובה עלי,אהובה עליי,אהיה,אובדן,אוגוסט,אודה,אוהב,אוהב טייל,אוהב מאוד,אוהב מיוחד,אוהב עשה,אוהבת,אוהד,אוהל,אוויר,אוויר אחר
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,7,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1071,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1072,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1073,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0
1074,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [16]:
# cross validation
from sklearn.metrics import make_scorer

def cross_validation(clf, X, y):
    # 10 folds, average f1 score by default 
    scores = cross_val_score(clf, X, y, scoring=make_scorer(f1_score, average='macro'), cv=10)
    return scores

In [22]:
# list of classifiers to compare
classifiers = [
    svm.SVC(kernel='linear', random_state=42),
    LinearSVC(random_state=42),
    MLPClassifier(random_state=42),
    Perceptron(random_state=42),
    SGDClassifier(random_state=42),
    KNeighborsClassifier(),
    DecisionTreeClassifier(random_state=42),
    MultinomialNB(),
    GaussianNB()
]

# the parameter grid for each classifier
param_grids = [
    {'C': [0.1, 1, 10]},
    {'C': [0.1, 1, 10]},
    {'hidden_layer_sizes': [(100,), (50, 100), (100, 100)], 'alpha': [0.0001, 0.001, 0.01]},
    {'alpha': [0.0001, 0.001, 0.01]},
    {'alpha': [0.0001, 0.001, 0.01]},
    {'n_neighbors': [3, 5, 7]},
    {'max_depth': [2, 4, 6, 8, 10]},
    {},
    {}
]

scoring_metric = make_scorer(f1_score, average='macro')

for clf, param_grid in zip(classifiers, param_grids):
    grid_search = GridSearchCV(clf, param_grid, cv=10, scoring=scoring_metric, n_jobs=-1)
    grid_search.fit(X_train.toarray(), y_train)
    print('Best parameters for {}: {}'.format(clf.__class__.__name__, grid_search.best_params_))
    print('Best {} score for {}: {:.2f}'.format(scoring_metric, clf.__class__.__name__, grid_search.best_score_))

GridSearchCV(cv=10, estimator=SVC(kernel='linear', random_state=42), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10]},
             scoring=make_scorer(f1_score, average=macro))

Best parameters for SVC: {'C': 0.1}
Best make_scorer(f1_score, average=macro) score for SVC: 0.64


/home/matan/.local/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/matan/.local/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/matan/.local/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/matan/.local/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/matan/.local/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/matan/.local/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to conver

GridSearchCV(cv=10, estimator=LinearSVC(random_state=42), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10]},
             scoring=make_scorer(f1_score, average=macro))

Best parameters for LinearSVC: {'C': 1}
Best make_scorer(f1_score, average=macro) score for LinearSVC: 0.66


...

Best parameters for DecisionTreeClassifier: {'max_depth': 8}
Best make_scorer(f1_score, average=macro) score for DecisionTreeClassifier: 0.71


GridSearchCV(cv=10, estimator=MultinomialNB(), n_jobs=-1, param_grid={},
             scoring=make_scorer(f1_score, average=macro))

Best parameters for MultinomialNB: {}
Best make_scorer(f1_score, average=macro) score for MultinomialNB: 0.71


GridSearchCV(cv=10, estimator=GaussianNB(), n_jobs=-1, param_grid={},
             scoring=make_scorer(f1_score, average=macro))

Best parameters for GaussianNB: {}
Best make_scorer(f1_score, average=macro) score for GaussianNB: 0.48


### Now we will the 'DecisionTreeClassifier' upon the test set

In [38]:
# fit the model on all the training set
clf = DecisionTreeClassifier(max_depth=8, random_state=42)
clf.fit(X_train.toarray(), y_train)

# predict for the test_set
y_test_pred = clf.predict(X_test.toarray())

DecisionTreeClassifier(max_depth=8, random_state=42)

In [54]:
test_predicted = pd.DataFrame(y_test_pred, columns=['gender']).replace({0: 'm', 1: 'f'})
test_predicted.head()
test_predicted.tail()

,gender
0,m
1,m
2,m
3,m
4,m


,gender
318,m
319,m
320,f
321,m
322,m


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [ ]:
df_predicted.to_csv('classification_results.csv',index=False)